In [1]:
### XAI ----------------------------
# 버전 맞추기,,,,,,,
!pip install dice-ml
!pip install --force-reinstall numpy==1.23.5 pandas==1.5.3 scikit-learn==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 41.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
dask-expr 1.1.21 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2025.3.1 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you h

In [1]:
import pandas as pd
import numpy as np
import pickle
import joblib
import dice_ml
from dice_ml.utils import helpers
import xgboost as xgb
# 역변환
from scipy.special import inv_boxcox
### 모델링--------------------------
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, precision_score, f1_score, recall_score, accuracy_score, roc_auc_score
### 새로운 데이터 --------------------
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.preprocessing import LabelEncoder
### x 만들기 -----------------------
# 지오코딩
!pip install pandas openpyxl
!pip install geokakao
import geokakao as gk
# 역지오코딩
import requests
import re
# 배수등급, 경사도
import geopandas as gpd
from shapely.geometry import Point
# # 시각화
# import matplotlib.pyplot as plt
# plt.rc('font', family='NanumBarunGothic')
# plt.rcParams['axes.unicode_minus'] =False
# import matplotlib.font_manager as fm
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [2]:
# 데이터 & 모델 로드
pothole = pd.read_csv('/content/drive/MyDrive/DATA_완성/pothole_예측용.csv')
pothole['발생여부'] = np.where(pothole['포트홀 갯수'] >=1, 1, 0)

x = pothole[['차선수', '승용차', '버스', '트럭', '평균_건물연령', '인구 수', '배수등급', '경사도']]
x = x.astype('float64')
x_test = x[:10] ## 기존에 데이터 분리할 때 사용한 코드가 하나도 안 먹혀서,, 이렇게라도

### 모델 불러오기
model_path = '/content/drive/MyDrive/준희/Modeling/Prediction/xgb_model_wo_env.pkl'
xgb_model = joblib.load(model_path)

### transformer, scaler
transformer_path = '/content/drive/MyDrive/준희/XAI/BoxCoxTransformer_wo_env_dice.pkl'
scaler_path = '/content/drive/MyDrive/준희/XAI/StandardScaler_wo_env_dice.pkl'

In [3]:
### 역변환!!!! - Counterfactual에서 사용
def inverse_scaling_boxcox(transformed_data, transformer_path, scaler_path):

    # 역변환 대상 변수
    cols = ['승용차', '버스', '트럭', '평균_건물연령', '인구 수']
    cols_idx = [transformed_data.columns.get_loc(col) for col in cols]

    # scaler, transformer
    scaler = joblib.load(scaler_path)
    transformer = joblib.load(transformer_path)

    # 변환된 5개 변수만 추출
    transformed = transformed_data.iloc[:, cols_idx]

    # 역표준화
    un_scaled = scaler.inverse_transform(transformed)

    # 역변환
    un_transformed = np.array([inv_boxcox(un_scaled[:, i], transformer.lambdas_[i]) for i in range(un_scaled.shape[1])]).T

    # 전체 복원
    original = transformed_data.copy()
    for j, col in enumerate(cols):
        original[col] = un_transformed[:,j]


    ### 배수등급, 경사도 변수 처리
    # 경사도
    slope_encoding = {'0-2%': 0, '2-7%': 1, '7-15%': 2, '15-30%': 3, '30-60%': 4, '60-100%': 5}
    # 배수등급
    drain_encoding = {'매우양호': 5, '양호': 4, '약간양호': 3, '약간불량': 2, '불량': 1, '매우불량': 0}
    # 인코딩 딕셔너리 뒤집기
    slope_decoding = {v: k for k, v in slope_encoding.items()}
    drain_decoding = {v: k for k, v in drain_encoding.items()}

    # 역매핑
    original['경사도'] = original['경사도'].map(slope_decoding)
    original['배수등급'] = original['배수등급'].map(drain_decoding)

    return original

In [4]:
### DiCE 구성
continuous = ['승용차', '버스', '트럭', '평균_건물연령', '인구 수']
d = dice_ml.Data(dataframe = x_test.assign(발생여부 = 0),  # dummy target
                 continuous_features = continuous,
                 outcome_name = "발생여부")

m = dice_ml.Model(model = xgb_model, backend = "sklearn")
exp = dice_ml.Dice(d, m)

In [ ]:
### 예측이 1인 샘플 하나 골라서 counterfactual 생성
query_instance = x_test.iloc[[0]]
while xgb_model.predict(query_instance)[0] != 1:
    query_instance = x_test.sample(n=1)
# 값을 바꿀 변수들만 지정
features_to_change = ['승용차', '버스', '트럭', '평균_건물연령']

cf = exp.generate_counterfactuals(query_instance, total_CFs=1, desired_class="opposite",features_to_vary=features_to_change)
cf.visualize_as_dataframe()

100%|██████████| 1/1 [00:00<00:00,  4.30it/s]

Query instance (original outcome : 1)


,차선수,승용차,버스,트럭,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,1.0,-0.355737,-0.057007,-0.295055,-0.363365,0.928563,4.0,2.0,1



Diverse Counterfactual set (new outcome: 0)


,차선수,승용차,버스,트럭,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,1.0,-1.052632,-0.057007,-0.295055,-0.363365,0.928563,4.0,2.0,0


In [ ]:
### 데이터프레임 저장
# 기존 데이터
original_one = cf.cf_examples_list[0].test_instance_df
# 바뀐 데이터
changed_zero = cf.cf_examples_list[0].final_cfs_df

In [ ]:
original_one = inverse_scaling_boxcox(original_one, transformer_path, scaler_path)
changed_zero = inverse_scaling_boxcox(changed_zero, transformer_path, scaler_path)

In [ ]:
original_one

,차선수,승용차,버스,트럭,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,1.0,15805.000064,687.000001,1721.999995,30.813624,237377.882668,양호,7-15%,1


In [ ]:
changed_zero

,차선수,승용차,버스,트럭,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,1.0,14746.768879,687.000001,1722.000001,30.813624,237377.882354,양호,7-15%,0


- 해석
  - 승용차 교통량이 저만큼 줄면 된다~
  - 트럭은 오히려 조금 늘었지만 괜찮았다~

### 최종 함수 정리

In [ ]:
### 역변환!!!! - Counterfactual에서 사용
def inverse_scaling_boxcox(transformed_data, transformer_path, scaler_path):

    # 역변환 대상 변수
    cols = ['승용차', '버스', '트럭', '평균_건물연령', '인구 수']
    cols_idx = [transformed_data.columns.get_loc(col) for col in cols]

    # scaler, transformer
    scaler = joblib.load(scaler_path)
    transformer = joblib.load(transformer_path)

    # 변환된 5개 변수만 추출
    transformed = transformed_data.iloc[:, cols_idx]

    # 역표준화
    un_scaled = scaler.inverse_transform(transformed)

    # 역변환
    un_transformed = np.array([inv_boxcox(un_scaled[:, i], transformer.lambdas_[i]) for i in range(un_scaled.shape[1])]).T

    # 전체 복원
    original = transformed_data.copy()
    for j, col in enumerate(cols):
        original[col] = un_transformed[:,j]


    ### 배수등급, 경사도 변수 처리
    # 경사도
    slope_encoding = {'0-2%': 0, '2-7%': 1, '7-15%': 2, '15-30%': 3, '30-60%': 4, '60-100%': 5}
    # 배수등급
    drain_encoding = {'매우양호': 5, '양호': 4, '약간양호': 3, '약간불량': 2, '불량': 1, '매우불량': 0}
    # 인코딩 딕셔너리 뒤집기
    slope_decoding = {v: k for k, v in slope_encoding.items()}
    drain_decoding = {v: k for k, v in drain_encoding.items()}

    # 역매핑
    original['경사도'] = original['경사도'].map(slope_decoding)
    original['배수등급'] = original['배수등급'].map(drain_decoding)

    return original

In [5]:
def dice(x_test, model_path, transformer_path, scaler_path): # x_test : 예측이 1로 나오는 예시 샘플 하나
    xgb_model = joblib.load(model_path)

    # 혹시 모르니까 한 번 더 변환해주고
    x_test = x_test.astype('float64')

    ### DiCE
    continuous = ['승용차', '버스', '트럭', '평균_건물연령', '인구 수']
    d = dice_ml.Data(dataframe = x_test.assign(발생여부 = 0),  # dummy target
                     continuous_features = continuous,
                     outcome_name = "발생여부")
    m = dice_ml.Model(model = xgb_model, backend = "sklearn")
    exp = dice_ml.Dice(d, m)

    ### counterfactual 생성
    # 값을 바꿀 변수들만 지정
    features_to_change = ['승용차', '버스', '트럭', '평균_건물연령']
    query_instance = x_test
    cf = exp.generate_counterfactuals(query_instance, total_CFs=1, desired_class="opposite",features_to_vary=features_to_change)

    ### 데이터프레임 저장
    # 기존 데이터
    original_one = cf.cf_examples_list[0].test_instance_df
    # 바뀐 데이터
    changed_zero = cf.cf_examples_list[0].final_cfs_df

    original_one = inverse_scaling_boxcox(original_one, transformer_path, scaler_path)
    changed_zero = inverse_scaling_boxcox(changed_zero, transformer_path, scaler_path)

    return original_one, changed_zero

In [7]:
x_test = pd.read_csv("/content/drive/MyDrive/준희/Report/Files/new_places_x_pres.csv")
x_test.drop('Unnamed: 0', axis=1, inplace=True)


In [8]:
original_one, changed_zero = dice(x_test, model_path, transformer_path, scaler_path)

  0%|          | 0/1 [00:00<?, ?it/s]


ValueError: feature_names mismatch: ['차선수', '승용차', '버스', '트럭', '평균_건물연령', '인구 수', '배수등급', '경사도'] ['차선수', '승용차', '버스', '트럭', '총교통량', '중대형차량 교통량', '평균_건물연령', '인구 수', '배수등급', '경사도']
training data did not have the following fields: 총교통량, 중대형차량 교통량